In [1]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import pprint
from bson.objectid import ObjectId
import pyperclip as clip
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools as it
import itertools
import time
# from datasu import auc
# import pixiedust
# clip.copy(json.dumps(m1['msg']))
import nbimporter
from common_functions import *
from nbimporter import NotebookLoader
loader = NotebookLoader()
from common_functions_message import *
Message = loader.load_module("common_functions_message").Message

pd.options.mode.chained_assignment = None  # default='warn'

import pixiedust

Importing Jupyter notebook from common_functions.ipynb
Importing Jupyter notebook from common_functions_message.ipynb
Importing Jupyter notebook from common_functions_message.ipynb
Pixiedust database opened successfully


### funcs

In [2]:
def create_emb_document(timestamp, genie_message_id, genie_conversation_id, message_index, msg_vector, msg_rpr_key, rpr_metadata, enc_metadata):    
    msg_rpr_metadata = {
        'shape': list(msg_vector.shape),
        'msg_rpr': rpr_metadata,
    }
    
    emb_doc = {        
        'genie_conversation_message_id': genie_message_id, 
        'genie_conversation_id':genie_conversation_id,
        'message_index': message_index,    
        'rpr_key': msg_rpr_key,
        'version': timestamp,
        
        'msg_vector': msg_vector.tolist(),       
        
        'rpr_metadata': msg_rpr_metadata,        
        'enc_metadata': enc_metadata
    }
    return emb_doc

In [5]:
def embed_genie_messages(db, msg_chunk_iterator, msg_rpr_key, rpr_metadata, enc_metadata, msgs_to_vector_delegate):
    chunk_n=0
    total_docs=0
    # ppp = []

    timestamp = time.time()

    for docs in msg_chunk_iterator:
        chunk_n=chunk_n+1        
        chunk_len = 0   
        msgs = list(docs)

        emb_vectors = msgs_to_vector_delegate(msgs)

        def de(msg, msg_vector):
            return create_emb_document(timestamp, msg['_id'], msg['conversation_id'], msg['message_index'], msg_vector, msg_rpr_key, rpr_metadata, enc_metadata)        
        msg_emb_docs = [de(msg, msg_vector) for msg,msg_vector in zip(msgs, emb_vectors)]

        for emb_doc in msg_emb_docs:
            f = {
                'genie_conversation_message_id': emb_doc['genie_conversation_message_id'],
                'genie_conversation_id': emb_doc['genie_conversation_id'],
                'message_index': emb_doc['message_index'],
                'rpr_key': emb_doc['rpr_key'],            
            }
            db.genie_conversation_messages_embs.replace_one(f, emb_doc, upsert=True)

    #         db_local.genie_conversation_messages_embs.insert_many(msg_emb_docs)

        chunk_len=chunk_len+len(msg_emb_docs)
        total_docs=total_docs+len(msg_emb_docs)
        print(f'chunk #: {chunk_n}, chunk_len: {chunk_len}')

    print("total docs iterated: ", total_docs)

### load data

In [6]:
# MONGO_HOST = "ec2-52-23-187-115.compute-1.amazonaws.com"
MONGO_DB = "marketpulzz"
# server = SSHTunnelForwarder(
#     MONGO_HOST,
#     ssh_username='ubuntu',
#     ssh_pkey="/Users/sasha/.ssh/mmuze.pem",
# #     ssh_private_key_password="secret",
#     remote_bind_address=('127.0.0.1', 27017),
#     local_bind_address=('127.0.0.1', 63328),
#     set_keepalive = 5,
# )

# server.start()
# client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
# db = client[MONGO_DB]

In [7]:
client_local = MongoClient('127.0.0.1', 27017)
db_local = client_local[MONGO_DB]

In [8]:
projection={'msg.text':1, '_id':1, 'conversation_id':1,'message_index':1}
skip=0
# genie_conversation_messages_cursor = db_local.genie_conversation_messages.find({}, projection=projection)[skip:]

### embed messages

#### USE

In [9]:
# https://stackoverflow.com/questions/48092772/add-operation-to-graph-without-with-as-clause
import tensorflow_hub as hub
graph = tf.Graph()
cm = graph.as_default()   
cm.__enter__()
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 
model_USE = 'universal-sentence-encoder-large/3'
module_url = f"https://tfhub.dev/google/{model_USE}" 
#@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url, trainable=True)

session = tf.Session(graph=graph)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
input1 = tf.placeholder(tf.string, shape=(None))
emb = embed(input1)


def encode_USE(texts):   
    emb1 = session.run([emb], feed_dict={ input1: texts })
    return emb1

#warm up
_ = encode_USE(["sasha", "miko"])

INFO:tensorflow:Using /var/folders/xn/dh4vn2gx4x36vh0x051kpw1m0000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [25]:
def _embed_messages_text_use_1(msg_bsons):    
    texts = list(map(lambda msg: msg['msg']['text'], msg_bsons))    
    emb_USE = encode_USE(texts)[0]     
    return emb_USE
    

In [26]:
use_metadata = {    
    'encoder': 'USE',    
    'model': model_USE,    
}
rpr_metadata = {
    'fields': [{'field':'msg.text', 'emb':'emb(msg.text)'}],    
}


In [32]:
text_len_filter = {'$where': 'this.msg.text.length > 1'}
query={**text_len_filter}

In [33]:
text_len_filter

{'$where': 'this.msg.text.length > 1'}

In [38]:
mess_chunk_iter = iterate_by_chunks(db_local.genie_conversation_messages, chunksize=100, start_from=0, query=query, projection=projection)

In [28]:
embed_genie_messages(db_local, mess_chunk_iter, 'text.use.1', rpr_metadata, use_metadata, _embed_messages_text_use_1)

chunk #: 1, chunk_len: 100
chunk #: 2, chunk_len: 100
chunk #: 3, chunk_len: 100
chunk #: 4, chunk_len: 100
chunk #: 5, chunk_len: 100
chunk #: 6, chunk_len: 100
chunk #: 7, chunk_len: 100
chunk #: 8, chunk_len: 100
chunk #: 9, chunk_len: 100
chunk #: 10, chunk_len: 100
chunk #: 11, chunk_len: 100
chunk #: 12, chunk_len: 100
chunk #: 13, chunk_len: 100
chunk #: 14, chunk_len: 100
chunk #: 15, chunk_len: 100
chunk #: 16, chunk_len: 100
chunk #: 17, chunk_len: 100
chunk #: 18, chunk_len: 100
chunk #: 19, chunk_len: 100
chunk #: 20, chunk_len: 100
chunk #: 21, chunk_len: 100
chunk #: 22, chunk_len: 100
chunk #: 23, chunk_len: 100
chunk #: 24, chunk_len: 100
chunk #: 25, chunk_len: 100
chunk #: 26, chunk_len: 100
chunk #: 27, chunk_len: 81
total docs iterated:  2681


#### BERT

In [16]:
#bert-serving-start -model_dir /Users/sasha/dev/mmuze/_models/BERT/uncased_L-12_H-768_A-12/ -num_worker=1 -max_batch_size 16 -max_seq_len 200 -pooling_strategy REDUCE_MEAN -pooling_layer -2 
bert_metadata = {            
    'version': '1.0',
    'encoder': 'BERT',
    'model': 'uncased_L-12_H-768_A-12',    
    'max_seq_len': 200,
    'pooling_strategy': 'REDUCE_MEAN', 
    'pooling_layer': '-5',
#     'dimensions': emb_bert[0].shape[0]
}

print(f"bert-serving-start \
-model_dir /Users/sasha/dev/mmuze/_models/BERT/{bert_metadata['model']} \
-num_worker=1 \
-max_batch_size 16 \
-max_seq_len {bert_metadata['max_seq_len']} \
-pooling_strategy {bert_metadata['pooling_strategy']} \
-pooling_layer {bert_metadata['pooling_layer']}")

bert-serving-start -model_dir /Users/sasha/dev/mmuze/_models/BERT/uncased_L-12_H-768_A-12 -num_worker=1 -max_batch_size 16 -max_seq_len 200 -pooling_strategy REDUCE_MEAN -pooling_layer -5


In [19]:
from bert_serving.client import BertClient
bc = BertClient()

In [18]:
rpr_metadata = {
    'fields': [{'field':'msg.text', 'emb':'emb(msg.text)'}],    
}



In [20]:
def _embed_messages_text_bert_1(msg_bsons):    
    texts = list(map(lambda msg: msg['msg']['text'], msg_bsons))    
    emb_bert = bc.encode(texts) 
    return emb_bert

In [37]:
mess_chunk_iter = iterate_by_chunks(db_local.genie_conversation_messages, chunksize=100, start_from=0, query=query, projection=projection)

In [24]:
embed_genie_messages(mess_chunk_iter, 'text.bert.1', rpr_metadata, use_metadata, _embed_messages_text_bert_1)

chunk #: 1, chunk_len: 100
chunk #: 2, chunk_len: 100
chunk #: 3, chunk_len: 100
chunk #: 4, chunk_len: 100
chunk #: 5, chunk_len: 100
chunk #: 6, chunk_len: 100
chunk #: 7, chunk_len: 100
chunk #: 8, chunk_len: 100
chunk #: 9, chunk_len: 100
chunk #: 10, chunk_len: 100
chunk #: 11, chunk_len: 100
chunk #: 12, chunk_len: 100
chunk #: 13, chunk_len: 100
chunk #: 14, chunk_len: 100
chunk #: 15, chunk_len: 100
chunk #: 16, chunk_len: 100
chunk #: 17, chunk_len: 100
chunk #: 18, chunk_len: 100
chunk #: 19, chunk_len: 100
chunk #: 20, chunk_len: 100
chunk #: 21, chunk_len: 100
chunk #: 22, chunk_len: 100
chunk #: 23, chunk_len: 100
chunk #: 24, chunk_len: 100
chunk #: 25, chunk_len: 100
chunk #: 26, chunk_len: 100
chunk #: 27, chunk_len: 81
total docs iterated:  2681


In [ ]:
%%time


In [ ]:
bert_emb_cache = list(map(lambda t: create_emb_cached_value(t[0],t[1].tolist(),bert_metadata) ,zip(texts, emb_bert)))

In [ ]:
for m in bert_emb_cache:
    db_local.embeddings_cache.save(m, check_keys=False)

In [ ]:
len(bert_emb_cache)

In [ ]:
z = get_embedding_from_cache('', use_metadata)